## The pure BERT model for token classification

In [1]:
! pip install seqeval==0.0.12 #BERT-base-cased

  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7434 sha256=c47bb8ef607626b7df3af01c3fa4fa2ae75397da176d86768224813a5897ed5c
  Stored in directory: /root/.cache/pip/wheels/dc/cc/62/a3b81f92d35a80e39eb9b2a9d8b31abac54c02b21b2d466edc
Successfully built seqeval


In [2]:
! pip install pytorch-transformers

     |████████████████████████████████| 176 kB 4.3 MB/s            


In [3]:
!pip install keras_preprocessing

In [4]:
! pip install scikit-learn

In [5]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import BertForTokenClassification, AdamW

In [6]:
df_train = pd.read_csv("/kaggle/input/jargon-detection/train.csv",sep=",",encoding="latin1")
df_train.head()

,token_id,token,label
0,cs/doc_1/0/0,A,O
1,cs/doc_1/0/1,SHAP,O
2,cs/doc_1/0/2,summary,O
3,cs/doc_1/0/3,plot,TERM
4,cs/doc_1/0/4,shows,O


In [7]:
df_val = pd.read_csv("/kaggle/input/jargon-detection/validation.csv",sep=",",encoding="latin1")
df_val.head()

,token_id,token,label
0,cs/doc_1867/0/0,Section,O
1,cs/doc_1867/0/1,~,O
2,cs/doc_1867/0/2,illustrates,O
3,cs/doc_1867/0/3,the,O
4,cs/doc_1867/0/4,proposed,O


In [8]:
df_test = pd.read_csv("/kaggle/input/jargon-detection/test_no_labels.csv",sep=",",encoding="latin1")
df_test.shape

(42358, 3)

In [9]:
df_val.label.unique()

array(['O', 'TERM'], dtype=object)

In [10]:
df_train[['m','n','x','y']] = df_train["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [11]:
df_train.label.value_counts()

O       520286
TERM     54624
Name: label, dtype: int64

In [12]:
df_val.label.value_counts()

O       33479
TERM     3664
Name: label, dtype: int64

In [13]:
df_test.label.value_counts()

O    42358
Name: label, dtype: int64

In [14]:
df_val[['m','n','x','y']] = df_val["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [15]:
df_test[['m','n','x','y']] = df_test["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [16]:
df_train = df_train.fillna(' ') 

In [17]:
df_val = df_val.fillna(' ') 

In [18]:
df_test = df_test.fillna(' ') 

In [19]:
len(df_test)

42358

In [20]:
df_train[['a','b']] = df_train["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_train

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1/0/0,A,O,cs,doc_1,0,0,doc,1
1,cs/doc_1/0/1,SHAP,O,cs,doc_1,0,1,doc,1
2,cs/doc_1/0/2,summary,O,cs,doc_1,0,2,doc,1
3,cs/doc_1/0/3,plot,TERM,cs,doc_1,0,3,doc,1
4,cs/doc_1/0/4,shows,O,cs,doc_1,0,4,doc,1
...,...,...,...,...,...,...,...,...,...
574905,phys/doc_1866/5/20,from,O,phys,doc_1866,5,20,doc,1866
574906,phys/doc_1866/5/21,a,O,phys,doc_1866,5,21,doc,1866
574907,phys/doc_1866/5/22,commercial,O,phys,doc_1866,5,22,doc,1866
574908,phys/doc_1866/5/23,supercapacitor,TERM,phys,doc_1866,5,23,doc,1866


In [21]:
df_val[['a','b']] = df_val["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_val

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1867/0/0,Section,O,cs,doc_1867,0,0,doc,1867
1,cs/doc_1867/0/1,~,O,cs,doc_1867,0,1,doc,1867
2,cs/doc_1867/0/2,illustrates,O,cs,doc_1867,0,2,doc,1867
3,cs/doc_1867/0/3,the,O,cs,doc_1867,0,3,doc,1867
4,cs/doc_1867/0/4,proposed,O,cs,doc_1867,0,4,doc,1867
...,...,...,...,...,...,...,...,...,...
37138,phys/doc_1976/10/28,vorticity,TERM,phys,doc_1976,10,28,doc,1976
37139,phys/doc_1976/10/29,in,O,phys,doc_1976,10,29,doc,1976
37140,phys/doc_1976/10/30,the,O,phys,doc_1976,10,30,doc,1976
37141,phys/doc_1976/10/31,wake,O,phys,doc_1976,10,31,doc,1976


In [22]:
df_test[['a','b']] = df_test["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_test

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1977/0/0,Next,O,cs,doc_1977,0,0,doc,1977
1,cs/doc_1977/0/1,",",O,cs,doc_1977,0,1,doc,1977
2,cs/doc_1977/0/2,we,O,cs,doc_1977,0,2,doc,1977
3,cs/doc_1977/0/3,analyze,O,cs,doc_1977,0,3,doc,1977
4,cs/doc_1977/0/4,the,O,cs,doc_1977,0,4,doc,1977
...,...,...,...,...,...,...,...,...,...
42353,phys/doc_2100/10/20,also,O,phys,doc_2100,10,20,doc,2100
42354,phys/doc_2100/10/21,presented,O,phys,doc_2100,10,21,doc,2100
42355,phys/doc_2100/10/22,and,O,phys,doc_2100,10,22,doc,2100
42356,phys/doc_2100/10/23,discussed,O,phys,doc_2100,10,23,doc,2100


In [23]:
df_train = df_train.drop('y', axis=1)
df_train = df_train.drop('token_id', axis=1)
df_train = df_train.drop('n', axis=1)
df_train = df_train.drop('a', axis=1)

In [24]:
df_val = df_val.drop('y', axis=1)
df_val = df_val.drop('token_id', axis=1)
df_val = df_val.drop('n', axis=1)
df_val = df_val.drop('a', axis=1)

In [25]:
df_test = df_test.drop('y', axis=1)
df_test = df_test.drop('token_id', axis=1)
df_test = df_test.drop('n', axis=1)
df_test = df_test.drop('a', axis=1)

In [26]:
df_train["s"] =  df_train["b"] + df_train['x'].astype(str) 

In [27]:
df_val["s"] = df_train["b"] + df_val['x'].astype(str)

In [28]:
df_test["s"] = df_test["b"] + df_test['x'].astype(str) 

In [29]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() 

In [30]:
df_train['c']= label_encoder.fit_transform(df_train['m'])

In [31]:
#Encode labels in column 'species'.
df_val['c']= label_encoder.fit_transform(df_val['m'])
df_val['m'].unique()

array(['cs', 'econ', 'phys'], dtype=object)

In [32]:
# Encode labels in column 'species'.
df_test['c']= label_encoder.fit_transform(df_test['m'])
df_test['m'].unique()

array(['cs', 'econ', 'phys'], dtype=object)

In [33]:
df_test["s"] = df_test['c'].astype(str) + df_test["s"]

In [34]:
df_val["s"] = df_val['c'].astype(str) + df_train["s"]

In [35]:
df_train["s"] = df_train["c"].astype(str) + df_train["s"]

In [36]:
df_train = df_train.drop('x', axis=1)
df_train = df_train.drop('b', axis=1)
df_train = df_train.drop('c', axis=1)

In [37]:
df_val = df_val.drop('x', axis=1)
df_val = df_val.drop('b', axis=1)
df_val = df_val.drop('c', axis=1)

In [38]:
df_test = df_test.drop('x', axis=1)
df_test = df_test.drop('b', axis=1)
df_test = df_test.drop('c', axis=1)

In [39]:
class divider(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(t,m,l) for t,m,l in zip(s["token"].values.tolist(),
                                                           s["m"].values.tolist(),
                                                           s["label"].values.tolist())]
        self.grouped = self.data.groupby("s").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [40]:
getter_trn = divider(df_train)
sentences_trn = [[s[0] for s in sent] for sent in getter_trn.sentences]


In [41]:
getter_val = divider(df_val)
sentences_val = [[s[0] for s in sent] for sent in getter_val.sentences]


In [42]:
getter_tst = divider(df_test)
sentences_tst = [[str(s[0]) for s in sent] for sent in getter_tst.sentences]

In [43]:
topic_trn = [[s[1] for s in sent] for sent in getter_trn.sentences]
labels_trn = [[s[2] for s in sent] for sent in getter_trn.sentences]

In [44]:
topic_val = [[s[1] for s in sent] for sent in getter_val.sentences]
labels_val = [[s[2] for s in sent] for sent in getter_val.sentences]

In [45]:
topic_tst = [[s[1] for s in sent] for sent in getter_tst.sentences]
labels_tst = [[s[2] for s in sent] for sent in getter_tst.sentences]

In [46]:
tags_vals = list(set(df_train["label"].values))

In [47]:
tags_vals = set(tags_vals)
tags_vals

{'O', 'TERM'}

In [48]:
tagtoid={'O': 0,
         'TERM' : 1,
         }

In [49]:
idtotag={tagtoid[key] : key for key in tagtoid.keys()}

In [50]:
device = torch.device("cuda" )if torch.cuda.is_available() else "cpu"
n_gpu = torch.cuda.device_count()

In [51]:
device

'cpu'

In [52]:
# Len of the sentence must be the same as the training model
# See model's 'max_position_embeddings' = 512
max_len  = 128
batch_num = 64
# load tokenizer, with manual file address or pretrained address

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

100%|██████████| 231508/231508 [00:00<00:00, 3880691.28B/s]


In [53]:
tokenized_texts_trn = []
word_piece_labels_trn = []

for word_list,label in (zip(sentences_trn,labels_trn)):
    temp_lable_trn = []
    temp_token_trn = []
    
    for word,lab in zip(word_list,label):
        temp_token_trn.append(word)
        temp_lable_trn.append(lab) 

    tokenized_texts_trn.append(temp_token_trn)
    word_piece_labels_trn.append(temp_lable_trn) 

In [54]:
tokenized_texts_trn[1]

['Although',
 'these',
 'models',
 'are',
 'typically',
 'considered',
 'unsupervised',
 'learning',
 'algorithms',
 ',',
 'since',
 'they',
 'are',
 'very',
 'sensitive',
 'to',
 'anomalies',
 '/',
 'outliers',
 'in',
 'the',
 'training',
 'set',
 '~',
 ',',
 'we',
 'have',
 'used',
 'them',
 'in',
 'a',
 'semi-supervised',
 'manner',
 '.']

In [55]:
tokenized_texts_val = []
word_piece_labels_val = []

for word_list,label in (zip(sentences_val,labels_val)):
    temp_lable_val = []
    temp_token_val = []
    
    for word,lab in zip(word_list,label):
        temp_token_val.append(word)
        temp_lable_val.append(lab)
           
    tokenized_texts_val.append(temp_token_val)
    word_piece_labels_val.append(temp_lable_val)  

In [56]:
tokenized_texts_val[0]

['Section',
 '~',
 'illustrates',
 'the',
 'proposed',
 'tilted',
 'layer',
 'fusion',
 'method',
 '.',
 'Section',
 '~',
 'shows',
 'the',
 'system']

In [57]:
tokenized_texts_tst = []
word_piece_labels_tst = []

for word_list,label in (zip(sentences_tst,labels_tst)):
    temp_lable_tst = []
    temp_token_tst = []
    
    for word,lab in zip(word_list,label):
        temp_token_tst.append(word)
        temp_lable_tst.append(lab)

    tokenized_texts_tst.append(temp_token_tst)
    word_piece_labels_tst.append(temp_lable_tst) 

In [58]:
tokenized_texts_tst[0]

['Next',
 ',',
 'we',
 'analyze',
 'the',
 'query',
 'frequency',
 'burst',
 '(',
 'cf.',
 'Figure',
 ')',
 '.']

In [59]:
input_ids_trn = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_trn],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_trn[0])

[  100   100 12654  5436  3065  1996   100  3643  1997  2838  2005  3265
  2951  2685  1012     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [60]:
input_ids_val = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_val],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_val[0])

[  100  1066 24899  1996  3818  9939  6741 10077  4118  1012   100  1066
  3065  1996  2291     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [61]:
input_ids_tst = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_tst],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_tst[0])

[  100  1010  2057 17908  1996 23032  6075  6532  1006   100   100  1007
  1012     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [62]:
input_ids_tst[15]

array([ 100, 1010,  100,  100, 2515, 2025, 8833, 3378,  100, 9537, 1010,
       2029, 2003, 6827, 2000, 2149, 1012,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [63]:
tags_trn = pad_sequences([[tagtoid.get(l) for l in lab] for lab in word_piece_labels_trn],
                     maxlen=max_len, value=tagtoid["O"], padding="post",
                     dtype="long", truncating="post")
print(tags_trn[0])

[0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [64]:
tags_val = pad_sequences([[tagtoid.get(l) for l in lab] for lab in word_piece_labels_val],
                     maxlen=max_len, value=tagtoid["O"], padding="post",
                     dtype="long", truncating="post")
print(tags_val[0])

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [65]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_trn = [[int(i>0) for i in ii] for ii in input_ids_trn]
attention_masks_trn[0];

In [66]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_val = [[int(i>0) for i in ii] for ii in input_ids_val]
attention_masks_val[0];

In [67]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_tst = [[int(i>0) for i in ii] for ii in input_ids_tst]
attention_masks_tst[0];

In [71]:
tr_inputs = torch.tensor(input_ids_trn)
val_inputs = torch.tensor(input_ids_val)
tst_inputs = torch.tensor(input_ids_tst)

tr_tags = torch.tensor(tags_trn)
val_tags = torch.tensor(tags_val)

tr_masks = torch.tensor(attention_masks_trn)
val_masks = torch.tensor(attention_masks_val)
tst_masks = torch.tensor(attention_masks_tst)

In [72]:
# Only set token embedding, attention embedding, no segment embedding
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [73]:
test_data = TensorDataset(tst_inputs, tst_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_num)

In [74]:
model = BertForTokenClassification.from_pretrained("bert-base-cased",num_labels=len(tagtoid))

100%|██████████| 440473133/440473133 [00:09<00:00, 44841393.18B/s]


In [75]:
model = model.to(device)

In [76]:
# Set epoch and grad max num
epochs = 3
max_grad_norm = 1.0

In [77]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [78]:
# Fine tune model all layer parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
    ]

In [79]:
optimizer = AdamW(optimizer_grouped_parameters, lr=0.0001)

In [80]:
# TRAIN loop
model.train();

## Training phase

In [81]:
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
Epoch:  33%|███▎      | 1/3 [22:06<44:12, 1326.15s/it]

Train loss: 0.13325234648823006


Epoch:  67%|██████▋   | 2/3 [44:04<22:01, 1321.52s/it]

Train loss: 0.09733266157149537


Epoch: 100%|██████████| 3/3 [1:06:03<00:00, 1321.16s/it]

Train loss: 0.07623080800693094


In [82]:
model_saving_address = 'models/final'
# Make dir if not exits
if not os.path.exists(model_saving_address):
    os.makedirs(model_saving_address)

In [83]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [84]:
# If we save using the predefined names, we can load using `from_pretrained`
model_file_location = os.path.join(model_saving_address, "pytorch_model.bin")
configuration_file_location = os.path.join(model_saving_address, "config.json")

# Save model into file
torch.save(model_to_save.state_dict(), model_file_location)
model_to_save.config.to_json_file(configuration_file_location)
tokenizer.save_vocabulary(model_saving_address)

('models/final/vocab.txt',)

In [85]:
model.eval();

## Evaluation phase

In [86]:
y_true = []
y_pred = []

for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label = batch
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,attention_mask=input_mask,)
        logits = outputs[0] 
   
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    label = label.to('cpu').numpy()
    
    input_mask = input_mask.to('cpu').numpy()
    
    for i,mask in enumerate(input_mask):
       # creating two empty lists.
        lst_1 = []
        lst_2 = []
        
        for j, m in enumerate(mask):
            if m:
                lst_1.append(idtotag[label[i][j]])
                lst_2.append(idtotag[logits[i][j]])
            else:
                break
        
            
        y_true.append(lst_1)
        y_pred.append(lst_2)
# print(y_pred)
# the report
report = classification_report(y_true, y_pred,digits=4)


print("\n%s"%(report))
# F1 score
print("Final F1 score: %f"%(f1_score(y_true, y_pred)))
# Accuracy Score
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))



           precision    recall  f1-score   support

     TERM     0.6985    0.7194    0.7088      3656

micro avg     0.6985    0.7194    0.7088      3656
macro avg     0.6985    0.7194    0.7088      3656

Final F1 score: 0.708799
Accuracy score: 0.941981


## Training phase

In [87]:
# Model we trained before, the dir containing pytorch_model.bin and vocab.txt
save_model_address = 'models/final'

In [88]:
save_model = BertForTokenClassification.from_pretrained(save_model_address,num_labels=len(tagtoid))

In [89]:
# Get model predict result
with torch.no_grad():
        outputs = save_model(tst_inputs, token_type_ids=None,attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        #logits = outputs[0] 

In [90]:
# Make logits into numpy type predict result
# The predict result contain each token's all tags predict result
predict_results = outputs[0].detach().cpu().numpy()

In [91]:
predict_results.shape

(1611, 128, 2)

In [92]:
from scipy.special import softmax

In [93]:
result_list = []
for pre_res in predict_results:
    result_arrays_soft =[]
    for i in pre_res:
        result_arrays_soft.append(softmax(i))
    result_list.append(np.argmax(result_arrays_soft,axis= -1))    

In [94]:
print(result_list[9])

[0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [95]:
print(input_ids_tst[15])

[ 100 1010  100  100 2515 2025 8833 3378  100 9537 1010 2029 2003 6827
 2000 2149 1012    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [96]:
print(tst_masks[9])

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


In [98]:
answer = []
for j in range(len(result_list)): 
    for i, mark in enumerate(tst_masks[j]):
        if mark>0:
            if result_list[j][i] == 1 :   
                answer.append('TERM')      
            else :   
                answer.append('O')

In [99]:
len(answer)

42358

In [100]:
submission_df = pd.read_csv("/kaggle/input/jargon-detection/test_no_labels.csv",sep=",",encoding="latin1")

In [101]:
submission_df['label'] = pd.DataFrame(answer)

In [102]:
submission_df = submission_df.drop(['token'], axis=1)

In [103]:
submission_df.to_csv("Bert_base_cased.csv", index=False)